## Data, Packages and Tools

The data used is the names of the hospitals, their locations, ratings and capacity, measured by number of beds. It comes principally from the following sources:

1. Canada 2016 Census, for selecting the most populated cities
2. Canadian Institute for Health Information (CIHI) for the number of beds per hospital
3. Forthsquare for Hospitals location and rating
4. Several python packages for managing and showing data, as pandas, folium and geopy. 
5. IBM Cloud Pak is the tool for running the code

### As an example a map of Canada showing the cities on the project is displayed

#### Importing the libraries that will be used in the project

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#conda install folium

!pip install folium
import folium # map rendering library
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

print('Libraries imported.') 

     |████████████████████████████████| 94 kB 6.7 MB/s  eta 0:00:01
Libraries imported.


#### Creating the data frame of the cities

In [2]:
address1 = 'Toronto, Ontario'
geolocator1 = Nominatim(user_agent="Toronto_explorer")
location1 = geolocator1.geocode(address1)
latitude1 = location1.latitude
longitude1 = location1.longitude
#print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude1, longitude1))

address2 = 'Montreal, Quebec'
geolocator2 = Nominatim(user_agent="Montreal_explorer")
location2 = geolocator2.geocode(address2)
latitude2 = location2.latitude
longitude2 = location2.longitude
#print('The geograpical coordinate of Montreal are {}, {}.'.format(latitude2, longitude2))

address3 = 'Calgary, Alberta'
geolocator3 = Nominatim(user_agent="Calgary_explorer")
location3 = geolocator3.geocode(address3)
latitude3 = location3.latitude
longitude3 = location3.longitude
#print('The geograpical coordinate of Calgary are {}, {}.'.format(latitude3, longitude3))

dfcanada = pd.DataFrame(np.array([["Toronto", "Ontario",2731571,latitude1,longitude1], ['Montreal','Quebec',1704694,latitude2,longitude2], ['Calgary','Alberta',1239220,latitude3,longitude3]]),
                   columns=['City', 'Province',"Population",'Latitude', 'Longitude'])
dfcanada

,City,Province,Population,Latitude,Longitude
0,Toronto,Ontario,2731571,43.6534817,-79.3839347
1,Montreal,Quebec,1704694,45.4972159,-73.6103642
2,Calgary,Alberta,1239220,51.0534234,-114.0625892


#### Generating Canadas map and marking the cities

In [3]:
# Map of Canada using latitude and longitude values
map_canada = folium.Map(location=[60.130, -110.35], zoom_start=4)

# add markers to map
for city, province, pop, lat, lng, in zip(dfcanada['City'], dfcanada['Province'], dfcanada['Population'], dfcanada['Latitude'], dfcanada['Longitude']):
    label = '{} ,{} ,{}'.format(city, province, pop)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='turquoise',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_canada)  
    
map_canada

In [4]:
#credentials for the Foursquare API
CLIENT_ID = 'KHLA0MRHSLCYLHZSLZAGPQX4QZL5NYKCFQSMRG2JAXUIWC2O' # your Foursquare ID
ACCESS_TOKEN = 'DR4VOEHRB3NBWSSFJ0NEOYB43MNJ1SVDO1DQCXJ4C3J1C0JU' # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 200
print('Your CLIENT_ID is: ' + CLIENT_ID)

Your CLIENT_ID is: KHLA0MRHSLCYLHZSLZAGPQX4QZL5NYKCFQSMRG2JAXUIWC2O


In [5]:
# @hidden_cel
CLIENT_SECRET = 'LMFK1NJLDAGTPRSNPMHOOANCZ55WFZYWLISDH0NLUSAWBDDO' # your Foursquare Secret

#### Exercise using Toronto Data

In [6]:
search_query = 'Hospital'
radius = 500000
print(search_query + ' .... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude1, longitude1,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)
url

Hospital .... OK!


'https://api.foursquare.com/v2/venues/search?client_id=KHLA0MRHSLCYLHZSLZAGPQX4QZL5NYKCFQSMRG2JAXUIWC2O&client_secret=LMFK1NJLDAGTPRSNPMHOOANCZ55WFZYWLISDH0NLUSAWBDDO&ll=43.6534817,-79.3839347&oauth_token=DR4VOEHRB3NBWSSFJ0NEOYB43MNJ1SVDO1DQCXJ4C3J1C0JU&v=20180604&query=Hospital&radius=500000&limit=200'

In [7]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '602055ac8a07476e5f824fee'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4ad4c064f964a5206ef820e3',
    'name': 'The Hospital for Sick Children (SickKids)',
    'location': {'address': '555 University Ave.',
     'crossStreet': 'at Gerrard St.',
     'lat': 43.657498668962646,
     'lng': -79.3865121609307,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.657498668962646,
       'lng': -79.3865121609307}],
     'distance': 492,
     'postalCode': 'M5G 1X8',
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['555 University Ave. (at Gerrard St.)',
      'Toronto ON M5G 1X8',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d196941735',
      'name': 'Hospital',
      'pluralName': 'Hospitals',
      'shortName': 'Hospital',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/medic

#### Get relevant part of JSON and transform it into a pandas dataframe

In [8]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()
#dataframe.shape

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress
0,4ad4c064f964a5206ef820e3,The Hospital for Sick Children (SickKids),"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1612731820,False,555 University Ave.,at Gerrard St.,43.657499,-79.386512,"[{'label': 'display', 'lat': 43.65749866896264...",492,M5G 1X8,CA,Toronto,ON,Canada,"[555 University Ave. (at Gerrard St.), Toronto..."
1,4ad4c064f964a5206ff820e3,St. Michael's Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1612731820,False,30 Bond St,at Queen St E,43.653784,-79.377809,"[{'label': 'display', 'lat': 43.65378419391942...",494,M5B 1W8,CA,Toronto,ON,Canada,"[30 Bond St (at Queen St E), Toronto ON M5B 1W..."
2,4ad4c064f964a52070f820e3,Toronto General Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1612731820,False,190 Elizabeth St,at Gerrard St W,43.658762,-79.388292,"[{'label': 'display', 'lat': 43.65876218215914...",684,M5G 2C4,CA,Toronto,ON,Canada,"[190 Elizabeth St (at Gerrard St W), Toronto O..."
3,4af0615cf964a5208cdb21e3,Women's College Hospital,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1612731820,False,76 Grenville St.,at Bay St.,43.661491,-79.387602,"[{'label': 'display', 'lat': 43.66149084521543...",939,M5S 1B2,CA,Toronto,ON,Canada,"[76 Grenville St. (at Bay St.), Toronto ON M5S..."
4,4e3dd18362e19d6109746a06,Toronto General Hospital Emergency Room,"[{'id': '4bf58dd8d48988d194941735', 'name': 'E...",v-1612731820,False,190 Elizabeth St,at Gerrard St W,43.658426,-79.386462,"[{'label': 'display', 'lat': 43.65842614181627...",586,NaN,CA,Toronto,ON,Canada,"[190 Elizabeth St (at Gerrard St W), Toronto O..."


In [9]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,The Hospital for Sick Children (SickKids),Hospital,555 University Ave.,at Gerrard St.,43.657499,-79.386512,"[{'label': 'display', 'lat': 43.65749866896264...",492,M5G 1X8,CA,Toronto,ON,Canada,"[555 University Ave. (at Gerrard St.), Toronto...",4ad4c064f964a5206ef820e3
1,St. Michael's Hospital,Hospital,30 Bond St,at Queen St E,43.653784,-79.377809,"[{'label': 'display', 'lat': 43.65378419391942...",494,M5B 1W8,CA,Toronto,ON,Canada,"[30 Bond St (at Queen St E), Toronto ON M5B 1W...",4ad4c064f964a5206ff820e3
2,Toronto General Hospital,Hospital,190 Elizabeth St,at Gerrard St W,43.658762,-79.388292,"[{'label': 'display', 'lat': 43.65876218215914...",684,M5G 2C4,CA,Toronto,ON,Canada,"[190 Elizabeth St (at Gerrard St W), Toronto O...",4ad4c064f964a52070f820e3
3,Women's College Hospital,Hospital,76 Grenville St.,at Bay St.,43.661491,-79.387602,"[{'label': 'display', 'lat': 43.66149084521543...",939,M5S 1B2,CA,Toronto,ON,Canada,"[76 Grenville St. (at Bay St.), Toronto ON M5S...",4af0615cf964a5208cdb21e3
4,Toronto General Hospital Emergency Room,Emergency Room,190 Elizabeth St,at Gerrard St W,43.658426,-79.386462,"[{'label': 'display', 'lat': 43.65842614181627...",586,NaN,CA,Toronto,ON,Canada,"[190 Elizabeth St (at Gerrard St W), Toronto O...",4e3dd18362e19d6109746a06


In [12]:
venues_map = folium.Map(location=[latitude1, longitude1], zoom_start=13) # generate map centred around Toronto
# add hospitals as turquoise circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map